In [6]:
import pandas as pd
import json

In [48]:
f = open('../raw/data/movie.json')
data = json.load(f)
result = [json.dumps(record) for record in data]
result = [json.dumps({"id": record['id'], 'title': record['title'], 'popularity': record['popularity']}) for record in data]
with open(f'./elasticmovie.json', 'w') as obj:
    for i in result:
        obj.write(i+'\n')

In [ ]:
df_movies = pd.read_json('../raw/data/movie.json')
df_movies.info()

In [ ]:
df_movies.head()

In [ ]:
print(df_movies.head(1))

In [12]:
from datetime import datetime, timedelta
import pytz
import random

def random_date(start, end):
    """
    Generate a random datetime between `start` and `end`.
    """
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

def format_datetime_to_iso(dt):
    """
    Format datetime to ISO 8601 format with 'Z' for UTC.
    """
    # Assuming the input datetime is naive (without timezone information)
    dt_utc = dt + timedelta(hours=-0)  # Manually adjust to UTC if needed
    return dt_utc.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + 'Z'

def conver_datetime(date_string):
    try:
        # Định dạng chuỗi ngày tháng ban đầu
        date_format_input = "%Y-%m-%d"

        # Chuyển đổi chuỗi ngày tháng thành đối tượng datetime
        date_object = datetime.strptime(date_string, date_format_input)

        # Add current time to the date object
        current_time = datetime.now().time()
        datetime_object = datetime.combine(date_object, current_time)

        # Convert to the desired format with milliseconds and UTC timezone
        utc_datetime_object = datetime_object.astimezone(pytz.utc)
        formatted_string = utc_datetime_object.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3] + 'Z'

        return formatted_string
    except Exception as ex:
        if type(date_string) == 'string' or date_string != '':
            date_return = date_string + 'T11:24:09.189Z'
            if date_return[0] == 'T':
                print(date_string)
            else:
                return date_return
        print(date_string)
        return '2014-06-25T11:24:09.189Z'

In [13]:
from datetime import datetime, date
def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))
def write_to_json(path, list_data):
    with open(path, 'w') as file:
        json.dump(list_data, file, indent=2, default=json_serial)
def checkStringEmpty(value):
    if value == "" or value is None:
        return True
    return False
def getLanguage(value):
    if value is None or len(value) == 0:
        return None
    return value[0]['name']
def getImageUrl(value):
    if checkStringEmpty(value):
        return None
    return 'https://image.tmdb.org/t/p/original' + value

In [ ]:
import random
from math import ceil

List_movie = []
# loop through the rows using iterrows()
for index, row in df_movies.iterrows():

    revenue_cus = row['revenue'] if row['revenue'] > 0 else random.randint(1000000, 2000000)
    budget_cus = row['budget'] if row['revenue'] > 0 else random.randint(50000, revenue_cus)

    vote_average = row['vote_average'] if row['vote_average'] > 0 else random.uniform(1.0, 10.0)
    vote_count = row['vote_count'] if row['vote_count'] > 0 else random.randint(20, 300)

    release_date = conver_datetime(row['release_date'])

    movie = {
        "id": row['id'],
        "title": row['title'],
        "genres": [genre['id'] for genre in row['genres']],
        "overview": row['overview'],
        "poster_path": getImageUrl(row['poster_path']),
        "backdrop_path": getImageUrl(row['backdrop_path']),
        "release_date": {"$date": conver_datetime(row['release_date'])},
        "popularity": row['popularity'],
        "revenue": revenue_cus,
        "budget": budget_cus,
        "status": row['status'],
        "vote_average": round(vote_average, 1),
        "vote_count": vote_count,
        "runtime": row['runtime'],
        "view": ceil(row['popularity'] * 10000 +  vote_count)
    }
    List_movie.append(movie)

write_to_json('./data/movie_db.json', List_movie)